2016-12-10 20:58:09

目标: 用 opencv 求图像序列外参, 图像序列组织形式按 TUM-dataset 规范

数据目录参考: @D:\Users\zhangxaochen\Documents\geo-cube399\cb5x8-cu399c-raw_frames-ts

近期背景: 使用 matlab APPs 套件, 内定标, 感觉内参比较靠谱; 但是外定标时, 有些图像角点都找不准, 反而比 opencv 差一些 (好的时候比 opencv 好)

参考代码:
1. depth-imu-calib.ipynb
2. depth-imu-calib-gen-synthetic.ipynb //以上关于外定标, 很好
3. compareEulerAngle.ipynb //暂没参考 2016-12-10 22:38:09

In [277]:
pwd

u'E:\\Github\\ipython.notebook.dir'

In [278]:
%pylab
import sys
sys.path.append(r'D:\opencv249\sources\modules\python\test')

import os
import cv2
import transformations as tf
'一些短别名:'
tf.e2m = tf.euler_matrix
tf.m2e = tf.euler_from_matrix
tf.q2m = tf.quaternion_matrix
tf.m2q = tf.quaternion_from_matrix
tf.e2q = tf.quaternion_from_euler
tf.q2e = tf.euler_from_quaternion
tf.m2r = tf.rotation_from_matrix
tf.r2m = tf.rotation_matrix

'angle <--> radian:'
a2r = np.pi / 180
r2a = 180 / np.pi

Using matplotlib backend: Qt4Agg
Populating the interactive namespace from numpy and matplotlib


In [279]:
'''
2016-12-11 17:16:24
拷贝自 compareEulerAngle.ipynb
'''
'some utils, 从下面 cell 剪切过来 //2016-5-13 10:16:00'

from lxml import etree
psr = etree.XMLParser(remove_blank_text=True)

def plotRvecs(fname):
    if fname.endswith('.csv'):
        pose_mat = loadtxt(fname, delimiter=',')
        rvecsDepth = pose_mat[:, 3:]
        return plot(rvecsDepth)
#     elif fname.endswith('.xml'):
#         tree = etree.parse(fname, parser=psr)
#         rt = tree.getroot()
#         tvecsColor = []
#         for i, iframe in enumerate(rt):
#             tvecsColor.append([float(num) for num in iframe.xpath('t/data')[0].text.split()])
#         tvecsColor = np.asarray(tvecsColor)
    pass

def plotTvecs(fname):
    if fname.endswith('.csv'):
        pose_mat = loadtxt(fname, delimiter=',')
        tvecsDepth = pose_mat[:, :3]
        return plot(tvecsDepth, '')
#     elif fname.endswith('.xml'):
#         tree = etree.parse(fname, parser=psr)
#         rt = tree.getroot()
#         tvecsColor = []
#         for i, iframe in enumerate(rt):
#             tvecsColor.append([float(num) for num in iframe.xpath('t/data')[0].text.split()])
#         tvecsColor = np.asarray(tvecsColor)/1000
#         plot(tvecsColor)
    pass

# def diffCsvs(fn1, fn2, minLen = inf):
# def diffCsvs(fn1, fn2, minLen = slice(None)):
def diffCsvs(fn1, fn2, rng = slice(None), minLen = inf): #主推 rng; 保留 minLen, 向后兼容
    extStr = '.csv'
    assert fn1.endswith(extStr) and fn2.endswith(extStr)
    '2 pose_mats:'
    m1 = loadtxt(fn1, delimiter=',')
    m2 = loadtxt(fn2, delimiter=',')
    '自动截断： 取两列表中短的为准'
    minLen = min(len(m1), len(m2), minLen) #若用户指定更小的 minLen, 则用它, 默认值 inf
    print 'diffCsvs(), len(m1), len(m2), minLen=', len(m1), len(m2), minLen
    
    m1 = m1[:minLen]
    m2 = m2[:minLen]
    m1 = m1[rng]
    m2 = m2[rng]
    
    print 'mean-abs-diff(各列):', np.abs(m1-m2).mean(axis=0)
    'tvecs:'
    m1tvecs = m1[:, :3]
    m2tvecs = m2[:, :3]

    'R: wxyz 序'
    m1rvecs = m1[:, 3:]
    m2rvecs = m2[:, 3:]
    m1rmats = np.array([tf.q2m([q[1], q[2], q[3], q[0]]) for q in m1rvecs])
    m2rmats = np.array([tf.q2m([q[1], q[2], q[3], q[0]]) for q in m2rvecs])
    
    '两序列起始帧: m1rmats[0], m2rmats[0] 未必一致, 所以要先处理成二者处于同一参考坐标系, 修改m2, 与m1对齐. 2016-5-26 22:35:05'
    m1t0 = m1tvecs[0]
    m1r0 = m1rmats[0]
    m2t0 = m2tvecs[0]
    m2r0 = m2rmats[0]
    
    '纸上推导: 2Ri = 1R0 * 2R0.T * 2Ri; 2ti = 1R0 * 2R0.T * (2ti-2t0) + 1t0; 2016-5-26 22:53:04'
    m2rmats = np.array([m1r0.dot(m2r0.T).dot(ri2) for ri2 in m2rmats])
    m2tvecs = np.array([m1r0.dot(m2r0.T)[:3,:3].dot(ti2-m2t0) + m1t0 for ti2 in m2tvecs])
    
    deltaQs = [tf.m2q(m1m.dot(m2m.T)) for m1m, m2m in zip(m1rmats, m2rmats)] #暂时没用上
    
    '绘制m1,m2 txyz'
    figure(figsize=(10,10))
    subplot(311)
    title('m1&m2-tvecs')
    m1ts = plot(m1tvecs)
    m2ts = plot(m2tvecs)
    legend(m1ts+m2ts, ['m1x', 'm1y', 'm1z', 'm2x', 'm2y', 'm2z'])

    '求解 ATE, 参照 evaluate_ate.py, 确信算法规则'
    #ate = norm(m1tvecs - m2tvecs) / sqrt(len(m1tvecs)) #可以, 但是下面改成也画每一帧误差, 所以不统着算
    tnorms = norm(m1tvecs - m2tvecs, axis=1) #每一帧二者的 tvec 偏差
    ate = norm(tnorms) / sqrt(len(m1tvecs))
    print 'ATE:', ate
    subplot(312)
    title('m1&m2-ATE')
#     plot(tnorms)
    plot(tnorms, '-r.') #带点折线, 非虚线
    for i in xrange(0, len(tnorms), 5):
        annotate('%.5f'%tnorms[i], (i, tnorms[i]))
#     ylim(0, 0.01)
    legend(['ATE=%.4fm'%ate], loc='lower right')
    return m1tvecs, m1rmats, m2tvecs, m2rmats
    pass

In [280]:
'''
2016-12-11 11:09:54
拷贝自 http://codepad.org/o1laMKIP, 见笔记《相机标定, 内定标, 通用》 1a3
intrMat 带 skew
distoMat 手填, 按 k1, k2, p1, p2, k3 顺序
'''
intrMat = np.array([
        [529.2196, -0.6437, 313.7732],
        [0,        528.9779, 254.0970],
        [0, 0, 1]])
distoMat = np.array([0.2377, -0.7297, 0.0015, 0.0028, 0.7367])

In [322]:
'''
2016-12-11 19:34:46
生成 rgb-cb-gt.csv, 以及 cv-cb-corner-draw 角点图目录 (注掉 cv2.imwrite 可不生成)
其中 csv: 无效行前7列全零, 其实可以不输出, 这里保留输出, 便于调试定位无效图像
'''

import cv2
import os, sys
from glob import glob

'数据proj根目录'
projDir = r'D:\Users\zhangxaochen\Documents\geo-cube399\cb5x8-cu399c-raw_frames-ts'
projDir = r'D:\Users\zhangxaochen\Documents\geo-cube399\cb5x8-cu399c-raw_frames-ts2'

assert os.path.exists(projDir), 'projDir: %s NOT exist~'%projDir
'对应这组数据的棋盘格内角点矩阵shape'
bsz = (5, 8)
'单格物理尺度mm'
bscale = (39.95, 39.43)

ptnPts = np.zeros( (np.prod(bsz), 3), np.float32)
ptnPts[:,:2] = np.indices(bsz).T.reshape(-1, 2) * bscale #mm 尺度

'depth、rgb关联描述文件, 这里可能用不到'
assoFn = os.path.join(projDir, 'zc-associate-res.txt')
assert os.path.exists(assoFn), 'assoFn: %s NOT exist~'%assoFn
rgbDir = os.path.join(projDir, 'rgb') #png dir
rgbPtn = os.path.join(rgbDir, '*.png') #pattern to glob
rgbFns = glob(rgbPtn)
'''
最终待存储的csv 2D数组, shape = N*8, 8列 = t3(m)+Q4+ts(float), //企图兼容 kinfu规范(前7列)
但是第8列还是没用上, //2016-12-12 01:19:51
'''
pose7 = zeros((len(rgbFns), 7))
pureFns = [0]*len(rgbFns) #随便拿 0 占位, 仅为了预分配内存

'找角点结果图, 存放在此路径, 以便观察调试'
# crnrImgDir = 'cv-cb-corner-draw'
crnrImgDir = os.path.join(projDir, 'cv-cb-corner-draw')
if not os.path.exists(crnrImgDir):
    os.makedirs(crnrImgDir)

for i, fn in enumerate(rgbFns):
#     if i>3:
#         break
    baseFn = os.path.basename(fn) #without folder
    '纯文件名, TUM格式中是纯时间戳'
    pureFn, ext = os.path.splitext(baseFn)
#     print pureFn 
    '无论下面是否正确求解, 第8列总要填, 若无解,则前7列全零'
#     pose7[i,7] = pureFn #不转 float, 保持 fixed 固定6位小数 
                        #错, np 自动转换 str->float
    #pureFns.append(pureFn)
    pureFns[i] = pureFn #改用预分配, 不用 append

    img = cv2.imread(fn)
    #cv2.undistort #是否不必要? 因为 solvePnp 有要求传入 disto 参数
    foundCrnr, crnrPxs = cv2.findChessboardCorners(img, bsz)
    if not foundCrnr:
        print 'foundCrnr FALSE, i: =  %d'%i
    else: #if foundCrnr, 进一步亚像素角点
        print i, 
        imgGray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        criteria  =  (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.01)
        cv2.cornerSubPix(imgGray, crnrPxs, (5,5), (-1,-1), criteria)
    viz = img.copy()
    cv2.drawChessboardCorners(viz, bsz, crnrPxs, foundCrnr)
    cv2.imshow('viz', viz)
    cv2.waitKey(1)
    
    '务必存档, 离线观察'
#     cv2.imwrite(os.path.join(crnrImgDir, 'cv-cb-corner-%s.png'%i), viz)
    
    #assert foundCrnr, 'foundCrnr FALSE, i: =  %d'%i
    if not foundCrnr:
        continue
    
    crnrPxs = crnrPxs.reshape(-1,2)
    ok, rvec, tvec = cv2.solvePnP(ptnPts, crnrPxs, intrMat, distoMat)
    #print 'ok:', ok, rvec, tvec #3*1， 3*1
    
    rmati = cv2.Rodrigues(rvec)[0]
    rmati4 = np.eye(4)
    rmati4[:3, :3] = rmati
    Qi = tf.m2q(rmati4) #记得: tf->xyzw 顺序, kinfu输出->wxyz顺序
    #pose7[i, :3] = tvec
#     pose7[i, :7] = np.hstack((tvec.ravel(), Qi[3], Qi[:3]))
    pose7[i] = np.hstack((tvec.ravel() / 1e3, Qi[3], Qi[:3])) #t: mm->m
csvres = hstack((pose7, reshape(pureFns, (-1, 1))) )
np.savetxt(os.path.join(projDir, 'rgb-cb-gt.csv'), csvres, delimiter=',', fmt='%s')#%.9f')

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187


In [287]:
'''
2016-12-11 17:16:53
回测观察 csv 长什么样
'''
# %pylab inline
m1fn=r'D:\Users\zhangxaochen\Documents\geo-cube399\cb5x8-cu399c-raw_frames-ts\rgb-cb-gt.csv'
lines=plotTvecs(m1fn)
legend(lines, list('xyz'))


In [316]:
a=np.array(['2', '10'])
a.sort()
a

array(['10', '2'], 
      dtype='|S2')

In [299]:
pose_mat = loadtxt(m1fn, delimiter=',')
tvecs = pose_mat[:, :3]
where(pose_mat[:,:7]==[0]*7)


(array([920, 920, 920, 920, 920, 920, 920]), array([0, 1, 2, 3, 4, 5, 6]))

In [319]:
[allclose(row[:7], [0]*7) for row in pose_mat][920:]

[True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 

In [311]:
a=eye(3)
b=array([True, False, True])
a[b]

array([[ 1.,  0.,  0.],
       [ 0.,  0.,  1.]])

In [310]:
y = np.arange(35).reshape(5,7)
b = y>20
y[b]

array([21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34])

In [318]:
'''
2016-12-11 17:14:41
一点测试代码, 测试: 角点, 外参, 亚像素, 棋盘格坐标系定义 
'''

fn=r'D:\Users\zhangxaochen\Documents\geo-cube399\cb5x8-cu399c-raw_frames-ts\rgb\1462501469.828000.png'
bsz=(5, 8)
'单格物理尺度mm'
bscale = (39.95, 39.43)
ptnPts = np.zeros( (np.prod(bsz), 3), np.float32)
ptnPts[:,:2] = np.indices(bsz).T.reshape(-1, 2) * bscale #mm 尺度 #宽-长-下
ptnPts[:,:2] = np.indices(bsz)[::-1].T.reshape(-1, 2) * bscale #长-宽-上
print 'ptnPts.shape: ', ptnPts.shape

img=cv2.imread(fn)
foundCrnr, crnrPxs=cv2.findChessboardCorners(img, bsz)
print 'crnrPxs.shape: ', crnrPxs.shape

imgCpy=img.copy()
cv2.drawChessboardCorners(imgCpy, bsz, crnrPxs, foundCrnr)
cv2.imshow('img', imgCpy)
cv2.waitKey()
imgGray=cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
cv2.imshow('imgGray', imgGray)
cv2.waitKey()
cv2.destroyAllWindows()
# cv2.cornerSubPix(imgGray, crnrPxs, (5,5), (-1,-1))
# imgGray=np.float32(imgGray)
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.01)
cv2.cornerSubPix(imgGray,crnrPxs,(5,5),(-1,-1),criteria)
# corners = cv2.cornerSubPix(img,crnrPxs,(11,11),(-1,-1),criteria)
cv2.drawChessboardCorners(imgCpy, bsz, crnrPxs, foundCrnr)
cv2.imshow('img', imgCpy)
cv2.waitKey()
cv2.destroyAllWindows()

ok, rvec, tvec = cv2.solvePnP(ptnPts, crnrPxs, intrMat, distoMat)
reprojPxs, jacob = cv2.projectPoints(ptnPts, rvec, tvec, intrMat, distoMat)
print 'reprojPxs.shape: ', reprojPxs.shape
cv2.drawChessboardCorners(imgCpy, bsz, reprojPxs[:prod(bsz)], foundCrnr)

imgCpy=img.copy()
ptnPtsAx = array([[0,0,0],[111,0,0],[0,111,0],[0,0,111]], float32) #注意必须指定 float32
reprojPxsAx, jacob = cv2.projectPoints(ptnPtsAx, rvec, tvec, intrMat, distoMat)
oo = tuple(reprojPxsAx[0].ravel())
xx = tuple(reprojPxsAx[1].ravel())
yy = tuple(reprojPxsAx[2].ravel())
zz = tuple(reprojPxsAx[3].ravel())
cv2.line(imgCpy, oo, xx, (255,0,0), 1)
cv2.line(imgCpy, oo, yy, (0,255,0), 2)
cv2.line(imgCpy, oo, zz, (0,0,255), 3)

cv2.imshow('img', imgCpy)
cv2.waitKey()
cv2.destroyAllWindows()

ptnPts.shape:  (40, 3)
crnrPxs.shape:  (40, 1, 2)
reprojPxs.shape:  (40, 1, 2)


上面测试代码得到的结果
--
棋盘格坐标系:
1. 原点右上角, 
2. 坐标轴向取决于板上平面上3D点 ptnPts 定义方式

>2a, np.indices(bsz).T.reshape(-1, 2) \* bscale

>[0,0],[1,0],...,[4,0], [0,1],[1,1],...
>![宽-长-下](https://i.stack.imgur.com/OP4Yf.png)

>2b, np.indices(bsz)[::-1].T.reshape(-1, 2) \* bscale

>[0,0],[0,1],...,[0,4], [1,0],[1,1],...
>![长-宽-上](https://i.stack.imgur.com/sadBe.png)

In [253]:
ptnPtsAx.dtype, ptnPts.dtype

(dtype('int32'), dtype('float32'))

In [248]:
xx, yy, zz

((313.74854, 235.49828), (334.24496, 226.45135), (354.08295, 217.68605))

In [241]:
tuple(oo.ravel())

(194.61314, 164.07445)

In [233]:
reprojPxs[:prod(bsz)].shape

(40, 1, 2)

In [217]:
imgCpy=img.copy()
for i, px in enumerate(crnrPxs2):
    if i>4:
        break
    cv2.circle(imgCpy, tuple(list(px)), 5, (0,0,255), 2)
cv2.imshow('img', imgCpy)
cv2.waitKey()

27